# 2유형 연습_타이타닉 생존자 분류

## 데이터 분석 순서

1. 라이브러리 및 데이터 확인
2. 데이터 탐색(EDA)
3. 데이터 전처리 및 분리
4. 모델링 및 성능평가
5. 예측값 제출

In [1]:
#### 복사 영역
# 실기 시험 데이터셋으로 세팅하기 (수정금지)
import pandas as pd
import numpy as np

import seaborn as sns
# titanic 데이터셋 로드
df = sns.load_dataset('titanic')

x = df.drop(['survived'], axis=1)
y = df['survived']

# 실기 시험 데이터셋으로 세팅하기 (수정금지)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2023)

x_test = pd.DataFrame(x_test)
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test) # 평가용


x_test.reset_index()
y_train.columns = ['target']
y_test.columns = ['target']

#### 참고사항 ####
# y_test는 주어지지 않음
# X를 대문자로 쓰지 말고 소문자 x로 쓸것. 타이핑할때 실수하기 쉽다.

### 타이타닉 생존자 예측 문제

- 데이터의 결측치, 중복 변수값에 대해 처리하고
- 분류모델을 사용하여 Accuracy, F1 score, AUC 값을 산출하시오.

데이터 설명
* survival : 0 = No, 1 = Yes
* pclass : 객실 등급(1, 2, 3)
* sex : 성별
* age : 나이
* sibsp : 탑승한 형제/배우자 수
* parch : 탑승한 부모/자녀의 수
* fare : 요금
* embarked : 탑승지 이름 Cherbourg / Queenstown / Southhampton
* (중복) class : 객실 등급(first, Second, Third)
* who : man, women, chind
* adult_male : 성인남자 True/False
* deck :  선실번호 알파벳(A~G)
* (중복) embark_town : 탑승지 이름 Cherbourg / Queenstown / Southhampton
* (중복) alive : 생존여부(no:사망, yes:생존)
* alone : 혼자 탑승했는지 여부(True=혼자, False=가족과 함께)

### 2. 데이터 탐색(EDA)

In [2]:
# 데이터의 행/열 확인
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(712, 14)
(179, 14)
(712, 1)


In [3]:
# 초기 데이터 확인
print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

     pclass     sex   age  sibsp  parch   fare embarked   class    who  \
3         1  female  35.0      1      0  53.10        S   First  woman   
517       3    male   NaN      0      0  24.15        Q   Third    man   
861       2    male  21.0      1      0  11.50        S  Second    man   

     adult_male deck  embark_town alive  alone  
3         False    C  Southampton   yes  False  
517        True  NaN   Queenstown    no   True  
861        True  NaN  Southampton    no  False  
     pclass     sex   age  sibsp  parch   fare embarked   class    who  \
800       2    male  34.0      0      0   13.0        S  Second    man   
341       1  female  24.0      3      2  263.0        S   First  woman   
413       2    male   NaN      0      0    0.0        S  Second    man   

     adult_male deck  embark_town alive  alone  
800        True  NaN  Southampton    no   True  
341       False    C  Southampton   yes  False  
413        True  NaN  Southampton    no   True  
     target
3 

In [4]:
# 타입 매칭, 결측치 확인
# object, category -> 원핫인코딩 필요

print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 3 to 608
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   pclass       712 non-null    int64   
 1   sex          712 non-null    object  
 2   age          579 non-null    float64 
 3   sibsp        712 non-null    int64   
 4   parch        712 non-null    int64   
 5   fare         712 non-null    float64 
 6   embarked     710 non-null    object  
 7   class        712 non-null    category
 8   who          712 non-null    object  
 9   adult_male   712 non-null    bool    
 10  deck         164 non-null    category
 11  embark_town  710 non-null    object  
 12  alive        712 non-null    object  
 13  alone        712 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(3), object(5)
memory usage: 64.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 800 to 410
Data columns (total 14 columns):
 #   Column       No

In [5]:
# x_train과 x_test 데이터의 기초통계량 비교
print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

           pclass         age       sibsp       parch        fare
count  712.000000  579.000000  712.000000  712.000000  712.000000
mean     2.307584   29.479568    0.518258    0.372191   31.741836
std      0.834926   14.355304    1.094522    0.792341   45.403910
min      1.000000    0.420000    0.000000    0.000000    0.000000
25%      2.000000   20.000000    0.000000    0.000000    7.895800
50%      3.000000   28.000000    0.000000    0.000000   14.454200
75%      3.000000   38.000000    1.000000    0.000000   31.275000
max      3.000000   74.000000    8.000000    6.000000  512.329200
           pclass         age       sibsp       parch        fare
count  179.000000  135.000000  179.000000  179.000000  179.000000
mean     2.312849   30.640741    0.541899    0.418994   34.043364
std      0.842950   15.258427    1.137797    0.859760   64.097184
min      1.000000    1.000000    0.000000    0.000000    0.000000
25%      2.000000   22.000000    0.000000    0.000000    7.925000
50%      3

In [6]:
# object, category 데이터도 추가 확인
print(x_train.describe(include='object'))
print(x_test.describe(include='object'))

print(x_train.describe(include='category'))
print(x_test.describe(include='category'))

         sex embarked  who  embark_town alive
count    712      710  712          710   712
unique     2        3    3            3     2
top     male        S  man  Southampton    no
freq     469      518  432          518   439
         sex embarked  who  embark_town alive
count    179      179  179          179   179
unique     2        3    3            3     2
top     male        S  man  Southampton    no
freq     108      126  105          126   110
        class deck
count     712  164
unique      3    7
top     Third    C
freq      391   47
        class deck
count     179   39
unique      3    7
top     Third    C
freq      100   12


In [7]:
# y데이터 살펴보기
print(y_train.head())

     target
3         1
517       0
861       0
487       0
58        1


In [8]:
print(y_train.value_counts())

target
0         439
1         273
Name: count, dtype: int64


### 3. 데이터 전처리 및 분리
1) 결측치
2) 이상치
3) 변수 처리하기

In [9]:
# 결측치 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

pclass           0
sex              0
age            133
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           548
embark_town      2
alive            0
alone            0
dtype: int64
pclass           0
sex              0
age             44
sibsp            0
parch            0
fare             0
embarked         0
class            0
who              0
adult_male       0
deck           140
embark_town      0
alive            0
alone            0
dtype: int64
target    0
dtype: int64


In [10]:
# 결측치 제거
# df = df.dropna()
# print(df)

# 참고사항
# print(df.dropna().shape) # 행 기준으로 삭제

In [11]:
# 결측치 대체
# x_train(712,14) : age(133), embarked(2), deck(548), embark_town(2)
# x_test(179,14) : age(44), deck(140)

# 변수 제거
# (중복) class, embark_town, alive
# (결측치 다수) deck -> 이정도의 숫자면 제거하는게 성능 향상

In [12]:
# 중복변수 제거
x_train = x_train.drop(['class', 'embark_town', 'alive', 'deck'], axis=1)
x_test = x_test.drop(['class', 'embark_town', 'alive', 'deck'], axis=1)

In [13]:
# 변수제거 확인
print(x_train.info())
print(x_test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 3 to 608
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      712 non-null    int64  
 1   sex         712 non-null    object 
 2   age         579 non-null    float64
 3   sibsp       712 non-null    int64  
 4   parch       712 non-null    int64  
 5   fare        712 non-null    float64
 6   embarked    710 non-null    object 
 7   who         712 non-null    object 
 8   adult_male  712 non-null    bool   
 9   alone       712 non-null    bool   
dtypes: bool(2), float64(2), int64(3), object(3)
memory usage: 51.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 800 to 410
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      179 non-null    int64  
 1   sex         179 non-null    object 
 2   age         135 non-null    float64
 3   sibsp       179 non

In [14]:
# 결측치 대체
# x_train(712, 14) : age(133), emarked(2)
# x_test(179, 14) : age(44)

# age 변수 -> 수치데이터는 중앙값으로 대체
med_age = x_train['age'].median()
x_train['age'] = x_train['age'].fillna(med_age)
x_test['age'] = x_test['age'].fillna(med_age) # train data의 중앙값으로

# embarked -> object 타입은 최빈값으로 대체
mode_et = x_train['embarked'].mode()
x_train['embarked'] = x_train['embarked'].fillna(mode_et[0]) # 최빈값 [0] 주의

In [15]:
# 결측치 대체 여부 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())

pclass        0
sex           0
age           0
sibsp         0
parch         0
fare          0
embarked      0
who           0
adult_male    0
alone         0
dtype: int64
pclass        0
sex           0
age           0
sibsp         0
parch         0
fare          0
embarked      0
who           0
adult_male    0
alone         0
dtype: int64


In [16]:
# 변수처리 (원핫인코딩)
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

print(x_train.info())
print(x_test.info())

# advanced 버전 사용
x_train_ad = x_train.copy()
x_test_ad = x_test.copy()
y_train_ad = y_train.copy()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 3 to 608
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      712 non-null    int64  
 1   age         712 non-null    float64
 2   sibsp       712 non-null    int64  
 3   parch       712 non-null    int64  
 4   fare        712 non-null    float64
 5   adult_male  712 non-null    bool   
 6   alone       712 non-null    bool   
 7   sex_female  712 non-null    bool   
 8   sex_male    712 non-null    bool   
 9   embarked_C  712 non-null    bool   
 10  embarked_Q  712 non-null    bool   
 11  embarked_S  712 non-null    bool   
 12  who_child   712 non-null    bool   
 13  who_man     712 non-null    bool   
 14  who_woman   712 non-null    bool   
dtypes: bool(10), float64(2), int64(3)
memory usage: 40.3 KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 179 entries, 800 to 410
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dt

In [17]:
# 참고사항 : 원핫인코딩 후 변수의 수가 다른 경우
# => x_test의 변수의 수가 x_train 보다 많은 경우 (혹은 그 반대인 경우)

# 원핫인코딩 후 Feature 수가 다를 경우
# x_train = pd.get_dummies(x_train)
# x_test = pd.get_dummies(x_test)
# x_train.info()
# x_test.info()

# 해결방법(x_test의 변수가 수가 더 많은 경우의 코드)
# x_train = x_train.reindex(columns = x_test.columns, fill_value=0)
# x_train.info()

데이터 분리

In [18]:
# 데이터 훈련 세트와 검증용 세트 분할 (8:2)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, 
                                                  y_train['target'], 
                                                  test_size=0.2, 
                                                  stratify=y_train['target'], 
                                                  random_state=2023)
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(569, 15)
(143, 15)
(569,)
(143,)


### 4. 모델링 및 성능평가

In [19]:
# 랜덤포레스트 모델 사용
from sklearn.ensemble import RandomForestClassifier
model  = RandomForestClassifier(random_state=2023)
model.fit(x_train, y_train)

RandomForestClassifier(random_state=2023)

In [20]:
# 모델을 사용하여 테스트 데이터 예측
y_pred = model.predict(x_val)

In [21]:
# 모델 성능 평가 (정확도, F1 score, 민감도, 특이도 등)
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score
acc = accuracy_score(y_val, y_pred) # (실제값, 예측값)
f1 = f1_score(y_val, y_pred)
auc = roc_auc_score(y_val, y_pred)

In [22]:
print(acc) # 정확도
print(f1) # f1 score
print(auc) # AUC

0.8531468531468531
0.8108108108108109
0.8465909090909092


In [23]:
# 참고사항 : 혼동행렬
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, y_pred)
print(cm)

[[77 11]
 [10 45]]


실제 test셋으로 성능평가를 한다면?

In [27]:
# 모델을 사용하여 테스트 데이터 예측
y_pred_f = model.predict(x_test)

# 모델 성능 평가 (정확도, F1 score, 민감도, 특이도 등)
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score
acc_f = accuracy_score(y_test, y_pred_f) # (실제값, 예측값)
f1_f = f1_score(y_test, y_pred_f)
auc_f = roc_auc_score(y_test, y_pred_f)

In [28]:
print(acc_f)
print(f1_f)
print(auc_f)

0.7821229050279329
0.7153284671532847
0.7687088274044797
